In [1]:
import os
import pickle
import os
import pickle

if os.path.exists('..\\data\\all_concepts_from_rake.pkl'):
# open the existing pickle file for reading
    with open('..\\data\\all_concepts_from_rake.pkl', 'rb') as f:
        all_concepts_from_rake = pickle.load(f)

In [22]:
######我加的
###############打印出来的格式是
# 得分：关键词
print(all_concepts_from_rake[:5]) # 如果是列表，打印前5个元素

[(33.0, 'reduced estimated total intracranial volume'), (29.0, 'dementia related socioeconomic'), (27.0, 'uncovering sex related pathways'), (27.0, 'normalized whole brain volume'), (26.0, 'resultslower education affected dementia')]


对RAKE算法提取的关键词进行后处理和频率统计，采用分批处理来解决内存溢出问题，最终结果：final_all_key_concepts_counter.pkl（包含所有关键词及其出现频率）

In [ ]:
# 内存溢出，拆开处理
import time
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer

# 初始化词形还原器
wnl = WordNetLemmatizer()

# 禁用字符
forbidden_chars = ['.', ',', '~', 'http', '&', '(', ')', '$', ' et al', '^', '{', '}', '/']


# 分批次处理函数
def process_batch(batch_start, batch_end, all_concepts_from_rake, all_key_concepts):
    # 处理一批数据
    for id_concept, curr_concept_with_score in enumerate(all_concepts_from_rake[batch_start:batch_end]):
        if curr_concept_with_score[0] > 1:  # 过滤掉得分小于1的概念
            curr_concept = curr_concept_with_score[1]

            if not any(char in curr_concept for char in forbidden_chars):
                curr_concept_split = curr_concept.split(' ')
                curr_concept_split[-1] = wnl.lemmatize(curr_concept_split[-1])
                curr_concept_split[-1] = curr_concept_split[-1].replace('matroids', 'matroid')
                curr_concept_singularized = ' '.join(curr_concept_split)
                all_key_concepts.append(curr_concept_singularized)
    return all_key_concepts


# 主要流程
def process_concepts_in_batches(all_concepts_from_rake, batch_size=100000):
    total_concepts = len(all_concepts_from_rake)
    all_key_concepts = []

    # 分批次处理
    for batch_start in range(0, total_concepts, batch_size):
        batch_end = min(batch_start + batch_size, total_concepts)
        print(f"Processing batch {batch_start} to {batch_end}...")

        # 处理当前批次
        all_key_concepts = process_batch(batch_start, batch_end, all_concepts_from_rake, all_key_concepts)

        # 每批次处理完后，保存当前结果
        partial_count = Counter(all_key_concepts)
        with open(f"partial_all_key_concepts_counter_{batch_start}.pkl", "wb") as output_file:
            pickle.dump(partial_count, output_file)

    # 最后合并所有批次结果
    final_count = Counter(all_key_concepts)
    with open("final_all_key_concepts_counter.pkl", "wb") as output_file:
        pickle.dump(final_count, output_file)


# 假设 all_concepts_from_rake 已经加载好
# 这里使用示例数据代替
#all_concepts_from_rake = [(2, "machine learning")]
process_concepts_in_batches(all_concepts_from_rake, batch_size=40000000)

In [4]:
## 原版
import time
import datetime
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from rake_nltk import Metric, Rake
from collections import Counter
from nltk.corpus import wordnet

# nltk.download('wordnet') Download necessary NLTK datasets
wnl=WordNetLemmatizer()
all_key_concepts=[]
forbidden_chars=['.',',','~','http','&','(',')','$', ' et al','^','{','}','/']

for id_concept, curr_concept_with_score in enumerate(all_concepts_from_rake):
    
    if curr_concept_with_score[0]>1:
        curr_concept=curr_concept_with_score[1]

        if (id_concept%2000000)==0:
            print(str(id_concept)+'/'+str(len(all_concepts_from_rake)))

        if not any(char in curr_concept for char in forbidden_chars):
            curr_concept_split=curr_concept.split(' ') 
            curr_concept_split[-1]=wnl.lemmatize(curr_concept_split[-1])
            curr_concept_split[-1]=curr_concept_split[-1].replace('matroids','matroid')
            curr_concept_singularized=' '.join(curr_concept_split)
            all_key_concepts.append(curr_concept_singularized)
        
all_concepts_count=Counter(all_key_concepts).most_common()
with open("all_key_concepts_counter.pkl", "wb") as output_file:
    pickle.dump(all_concepts_count, output_file)

0/79883107
2000000/79883107
4000000/79883107
6000000/79883107
8000000/79883107
10000000/79883107
12000000/79883107
14000000/79883107
16000000/79883107
18000000/79883107
20000000/79883107
22000000/79883107
24000000/79883107
26000000/79883107
28000000/79883107
30000000/79883107
32000000/79883107
34000000/79883107
36000000/79883107
38000000/79883107
40000000/79883107
42000000/79883107
44000000/79883107
46000000/79883107
48000000/79883107
50000000/79883107
52000000/79883107
54000000/79883107
56000000/79883107
58000000/79883107
60000000/79883107
62000000/79883107
64000000/79883107
66000000/79883107
68000000/79883107
70000000/79883107
72000000/79883107
74000000/79883107
76000000/79883107
78000000/79883107


In [5]:
import datetime
import pickle

# 从文件中加载 all_concepts_count
def load_concepts_count(file_path):
    with open(file_path, 'rb') as file:
        all_concepts_count = pickle.load(file)
    return all_concepts_count

# 读取 final_all_key_concepts_counter.pkl 中的数据
all_concepts_count = load_concepts_count("final_all_key_concepts_counter.pkl")

# 获取当前日期
curr_date = datetime.date.today()

# 打印概念计数的长度
print("all_concepts_count: ", len(all_concepts_count))


all_concepts_count:  29338139


筛选出高频3词+短语（≥6次），筛选出高频2词短语（≥9次）  

In [6]:
import datetime
import pickle

curr_date = datetime.date.today()
print("all_concepts_count: ", len(all_concepts_count))

full_concept_list = []

num3 = 6
num2 = 9

# 确保 all_concepts_count 是字典或者Counter，且使用 .items() 解包
for letter, count in all_concepts_count.items():  # 修改为 items() 来解包
    if len(letter.split(' ')) >= 3 and count >= num3:
        full_concept_list.append(letter)
print("3: ", len(full_concept_list))

# 第二轮筛选：长度为2，计数>= num2
for letter, count in all_concepts_count.items():  # 使用 items() 来解包
    if len(letter.split(' ')) == 2 and count >= num2:
        full_concept_list.append(letter)

print("total: ", len(full_concept_list))

# 保存筛选后的结果
with open(f"full_concepts_{num3}_{num2}.pkl", "wb") as output_file:
    pickle.dump(full_concept_list, output_file)

# 也将结果保存为文本文件
with open(f"full_concepts_{num3}_{num2}.txt", "a") as f:
    for ii in range(len(full_concept_list)):
        f.write(full_concept_list[ii] + '\n')


all_concepts_count:  29338139
3:  405954
total:  875840


### improve concepts

对关键词概念进行过滤和清洗，包含字符、文档格式标记和奇怪缩写


In [7]:
#########################这个函数未定义，我们自己写的###########################
def filter_words(concept, filter_list):
    """
    检查给定的概念（concept）是否包含在过滤字符列表（filter_list）中的任意字符。
    
    Parameters:
        concept (str): 需要检查的概念字符串。
        filter_list (list): 需要过滤的字符或词语列表。
        
    Returns:
        bool: 如果概念包含任何过滤字符，则返回 True，否则返回 False。
    """
    for filter_word in filter_list:
        if filter_word in concept:  # 如果概念中包含任何过滤词汇
            return False  # 返回 True 表示该概念不符合要求，应该移除
    return True  # 如果没有任何过滤词汇，返回 False，表示保留该概念


In [8]:
# starting_time = time.time()

filter_chars=['">','="','<"','>','<','=',']>','``','``\\','\\[','\\|','><','=\\','[\\',
              '"\\','\\%','|\\','+\\','\\`','\\!','?!','\\!\\','%;',']:',':\\','++']

weird_words=['o_fig','o_fig_display_l','no_fig','o_linksmallfig','o_tbl','_t','u_']


filter_chars.extend(weird_words)

new_reduced_list=[]
removed_list=[]
cc=0

for curr_concept in full_concept_list:
    
    cc+=1

    if (cc%20000)==0:
        print(str(cc)+'/'+str(len(full_concept_list)))

    if filter_words(curr_concept, filter_chars): #filter_words(curr_concept, filter_chars, filter_word_list)
        
        new_reduced_list.append(curr_concept)
    else:
        #print('remove : ', curr_concept)
        removed_list.append(curr_concept)

        
print("Concepts: {:d} ; Store: {:d}; Remove: {:d} ".format(len(full_concept_list), len(new_reduced_list),len(removed_list)))
# elapsed_time = time.time() - starting_time
# print("Elapsed time: {:.2f} seconds".format(elapsed_time))

20000/875840
40000/875840
60000/875840
80000/875840
100000/875840
120000/875840
140000/875840
160000/875840
180000/875840
200000/875840
220000/875840
240000/875840
260000/875840
280000/875840
300000/875840
320000/875840
340000/875840
360000/875840
380000/875840
400000/875840
420000/875840
440000/875840
460000/875840
480000/875840
500000/875840
520000/875840
540000/875840
560000/875840
580000/875840
600000/875840
620000/875840
640000/875840
660000/875840
680000/875840
700000/875840
720000/875840
740000/875840
760000/875840
780000/875840
800000/875840
820000/875840
840000/875840
860000/875840
Concepts: 875840 ; Store: 873992; Remove: 1848 


定义复杂的过滤规则体系
按词汇位置过滤
开头词过滤 (filter_start_words)：去除以"cause", "shows", "study"等开头的关键词

结尾词过滤 (filter_last_words)：去除以"show", "data", "class"等结尾的关键词

特定长度开头词 (filter_start_words_length)：对2词短语的特殊开头词过滤

按词汇类型过滤
通用停用词 (filter_lists)：描述性动词、形容词等（"described", "used", "better"）

副词过滤 (filter_other_adverb_lists)："successfully", "directly", "significantly"等

通用名词过滤 (filter_general_name_lists)："patients", "country", "school"等

时间单位过滤：月份、年份、计量单位等

In [9]:
import time
import re
def is_valid_year(string):
    pattern = re.compile(r'^\d{4}$')
    if pattern.match(string) is not None:
        year = int(string)
        return year <= 2025
    return False

starting_time = time.time()

#filter contains the word and also the length is two
filter_start_words_length=['fully','recall','sudden','change','least','ultimately','label','essentially','almost']

filter_start_words=['cause','performs','around','art','currently','current','sim','leq','carefully','simeq','giving','becomes',
                   'geqslant','geq','input','completely','000','highly','sigma','arbitrary','hence','study','actual',
                   'removing','ideal','fixed','theoretical','shows','estimates','correct','still','detailed','less','reduce',
                    'reduced','ensure','effectively','detect','deforming','draw','newly','following','manner','always','table',
                   'besides','italian','japanese','poor','although','federal','predicting','youtube','pregnant','relatively',
                   'gain','quite','particularly','actually','thereby','thoroughly','finally','perhaps','subsequently','existing',
                   'unlike','pushes','places','update','causes','bringing','similar','write','maximizing','generating','solve',
                    'indeed','spread','ask','truly','surprisingly','furthermore','additionally','break','search','eventually',
                    'leads','learning']



filter_last_words=['show', 'showed', 'showing','conference','caused','within','pdfs','employed','requirement','optical',
                   'km','data','alone','focus','sec','market','focused','collected','led','class','asked','larger','upgrade',
                   'increase','onto','m_','reflects','outcome','output','order','performs','ness','way','kev','solves','close',
                   'manner','according','developed','automatic','generalizes','relies','exceeds','aiming','dimensional','closely',
                  'exceeding','project','globally','smaller','input','u','richer','via','db','mm','governing','governed','like',
                   'analyzing','indicates','indicate','million','reveals','away','sigma','equivalent','linearly','seen','resolve',
                  'impossible','need','together','perspective','solve','one','growth','member','involving','generated','discovered',
                  'surely','along','ii','iii','driver','subtasks','company','province','council','woman','indicative','aim',
                   'label','us','start','try','go','almost','update','consists','forward','ahead','similar','spread','asks',
                   'observes','support','corroborate','insight','behind']


string_to_remove=['log log n','ra e','b ball','2 m_q','4 sca','high j','area 1','kappa 0','sim 2 3','0 ap',
               'least 1 1','2 coefficient','2 5 time','h2 0 0','4e 4']

filter_useless_strings=['computation cost','real world', 'real time', 'standard deviation', 'technical challenge',
                       ' times higher','high applied ','open source']


filter_words_fixed_len2=['support','strongly','jointly','start','independently','severe','go','iteratively','move','relevant',
                        'properly','empirically','main','dynamically','far','leading']

filter_lists=['described','describe','extends', 'studied', 'paper', 'shown', 'much', 'among', 'established', 'good', 'first', 'encodes', 'certain', 
              'increasing', 'various', 'used', 'unknown', 'lead','even', 'outperforms', 'ago', 'uses', 'entire', 'selected', 
              'last', 'within', 'present', 'yet', 'towards', 'based', 'well', 'achieve','achieves', 'achieved', 'regarding',
              'yields', 'review', 'may', 'better', 'performed', 'quot', 'learns', 'combined', 'practical','obtain', 'obtaining', 
              'patient', 'attains', 'year', 'vast', 'define', 'mev', 'provides', 'allows', 'generates', 'compared', 'reproducing',
              'improve', 'operates', 'years', 'trained', 'learn', 'would', 'beyond', 'derived', 'available', 'otherwise', 'every', 
              'best', 'result','resulting', 'make','makes', 'made', 'becoming', 'previous', 'arises', 'applying', 'obtained', 'poses', 
              'lacks', 'called', 'associated', 'proposes', 'brief', 'another', 'scores', 'promising', 'demonstrate', 'times',
              'challenging', 'act', 'suited', 'tremendous', 'beats', 'ral', 'work', 'multi', 'therefore', 'looking', 'important', 
              'extend', 'volunteer', 'could', 'recent', 'fit', 'produce', 'become', 'needed', 'providing', 'taken','turn',
               'observed', 'reviewer', 'phi', 'unless', 'apply', 'using', 'given', 'significant', 'proposed', 'published', 
               'give', 'sometimes', 'train', 'use', 'debate', 'applies', 'attracted', 'case', 'adding', 'includes', 'developed',
              'approximation','approx','approximate','whole','thus','characterize','surviving','degree','contain','relies','done',
              'determine','even','occurs','occur','avoid','estimate','estimated','reporting','task','hour','despite',
              'confirms','confirm','describing','release','progress','reported','several','run','running','degrees','derive',
              'derived','day','serious','introduce','introduced','institutional','without','evolve','deg','per','remain','remains',
              'proving','provide','office','university','subtask','appears','display','displayed','toward','takes','additional',
              'appendix','discuss','discussed','let','cannot','yield','gives','output','allowing','allow','june','submitted',
              'context','testing','often','requires','require','including','required','demonstrates','raised','enables',
              'making','help','helping','helps','tested','find','finding','gev','suggests','suggesting','construct','reformulated',
              'depends','reformulate','facilitating','future','might','ignored','ignore','typical','contributes','contribute',
              'revealed','included','investigate','investigated','investigates','different','next','excellent','faster','affected',
              'remarkable','expressed','found','improves','chosen','enable','cases','identify','modeled','suitable','utilizing',
              'predicted','predict','aged','healthy','exhibit','drew','justify','suggest','depend','behave','behaves','reproduces',
              'compute','happens','happen','calculate','perform','reduces','reduce','rise','unexpected','predicts','classify',
             'reproduce','vary','explore','analyze','explain','evaluate','affect','instead','works','bring','lack','puts','put',
             'remove','differs','differ','impose','imposes','develop','moves','consist','cause','brought','look','generate',
              'hold','holds','get','imply','illustrate','verify','implement','understood','understand','generalize','generalizes',
             'interpret','interprets','proven','follows','follow','interpret','acts','taking','becomes','account','observe','received',
             'know','reveal','reveals','validate','validates','agree','agrees','establish','creates','create','breaks','compare',
              'compares','consider','considers','yielding','yielded','offer','offers','provided','reaching']

filter_other_adverb_lists=['weekly','likely','unlikely','visually','successfully','previously','directly','indirectly','manually','seriously',
                   'explicitly','significantly','autonomously', 'approximately','optimally','widely','possibly','simultaneously',
                   'extremely','nearly','closely','largely','naturally','simply','solely','early','clearly','slightly','possible',
                  'typically','must','overall','either','unusually','usually','unusual','potential','potentially','rapidly','recently',
                   'rather','sufficiently','since','selectively','whether','appropriately','reliably','similarly','effectively','drastically',
                   'accurately','correctly','adequately','formally','systematically','efficiently','intuitively','exactly','together',
                    'due','everywhere','whereas','considerably','include','immediately','automatically','numerically','however',
                    'especially','poorly','particularly','easily','quickly','experimentally','briefly']

filter_general_name_lists=['german','russian','college','patients','country','month','workshop','participant','preprints',
                           'sections','percentage','section','table','version','institution','week','percent','minute','user',
                          'worker','students','investor','property','process','example','examples','million','bug','chapter',
                          'proposal','school','indian','european','american','government','chinese','city','uk','hospital',
                          'department','people','physician','github','presentation','english','license','python','summer',
                           'researchers','plan','researcher','months','days','players','results']

filter_months_lists = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
filter_units_lists=['myr','kv','kb','nm','wt','kg','au','kw','gk','mw','mv','ev','mhz','hz','khz','micron','cm2','fb','ghz','cm',
             'nu','gyr','pc','km','km2','kpc']

filter_symbol_lists=['%:','%]',"\\",'**','\\;\\',']\\',':[',']]',']+','\\\\','\\]']

filter_lists.extend(filter_other_adverb_lists)
filter_lists.extend(filter_general_name_lists)
filter_lists.extend(filter_months_lists)
filter_lists.extend(filter_units_lists)
filter_lists.extend(filter_symbol_lists)


improved_concept_list=[]
    
for one_concept in new_reduced_list:
    
    separated_words=one_concept.split()
    do_remove=0
    
    for word in separated_words:
        if word in filter_lists:
            do_remove=1
            break
        if is_valid_year(word):
            do_remove=1
            break            
            
        #for ii in range(len(filter_restricted1)):
            #if word==filter_restricted1[ii] and (filter_restricted2[ii] in separated_words):
                #do_remove=1
                #break
    
    for ii in range(len(filter_useless_strings)):
        if filter_useless_strings[ii] in one_concept:
            do_remove=1
            break

    if len(separated_words)==2:
        for word in separated_words:
            if word in filter_words_fixed_len2:
                do_remove=1
                break

    if separated_words[0].isnumeric() and len(separated_words)==2: # 2 books, 4 objects, etc...
        do_remove=1
    
       
    if separated_words[0] in filter_start_words:
        do_remove=1
            
    if separated_words[0] in filter_start_words_length and len(separated_words)==2 :
        do_remove=1
        
    if separated_words[-1] in filter_last_words:
        do_remove=1


    if do_remove==0:
        if one_concept not in string_to_remove:
            improved_concept_list.append(one_concept)
        
print("Concepts: {:d} ; Store: {:d}; Remove: {:d} ".format(len(new_reduced_list), len(improved_concept_list),len(new_reduced_list)-len(improved_concept_list)))
elapsed_time = time.time() - starting_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

print('\n')
now_time =  datetime.datetime.now()
formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
print("{}; Concepts: {:d} ".format(formatted_time,len(improved_concept_list)))



## here is just for a backup
f = open(f"full_concepts_stored_lists_{num3}_{num2}_improved.txt", "a")
for ii in range(len(improved_concept_list)):
    f.write(improved_concept_list[ii]+'\n')
f.close()
print("create text and store information.")


## here the stored concepts will be further improved
f = open(f"full_concepts_stored_lists_{num3}_{num2}_improved_modify.txt", "a")
for ii in range(len(improved_concept_list)):
    f.write(improved_concept_list[ii]+'\n')
f.close()
print("create text and store information.")

Concepts: 873992 ; Store: 613794; Remove: 260198 
Elapsed time: 12.31 seconds


05-12-2025 00:51:13; Concepts: 613794 
create text and store information.
create text and store information.


### further improve the concepts
load the txt file, filter concepts, remove the orginal txt file, re-store the txt file


1. 基础词汇过滤
数字相关：过滤包含数字字符或数字单词（one, two等）的概念

年份检测：使用正则表达式过滤纯年份概念

通用停用词：filter_concept_lists 包含大量描述性词汇

2. 字符串模式过滤
特定字符：过滤包含 *, :, +, ], ; 等符号的概念

短语模式：过滤常见无意义短语组合（"low cost", "high cost", "n x n"等）

复杂短语：filter_contain_str_lists 包含81个需要过滤的特定短语模式

3. 位置特定过滤
开头词限制：

通用开头词：filter_concept_start_words（187个）

长度相关开头词：filter_concept_start_restricted_len（73个）

结尾词限制：

通用结尾词：filter_concept_last_words（108个）

长度相关结尾词：filter_concept_last_restricted_len（36个）

第二词限制：对3词以上概念检查第二个词

4. 长度特定规则
2词概念特殊处理：

检查是否包含受限词汇

验证开头和结尾词

处理"单词+数字"模式

检查单词长度组合合理性

3词概念特殊处理：

处理"数字+数字+单词"模式

检查各位置单词长度组合

5. 高级语义过滤
重复结构检测：识别并过滤如"word word"这类重复结构

低分词汇检测：

开头低分词：如果去掉开头词后概念仍存在，则过滤原概念

结尾低分词：如果去掉结尾词后概念仍存在，则过滤原概念

复数形式处理：如果单数形式已存在，则过滤复数形式

In [10]:
if os.path.exists('full_concepts_stored_lists_6_9_improved_modify.txt'):
    # open the existing pickle file for reading   
    with open("full_concepts_stored_lists_6_9_improved_modify.txt", "r") as f:
        modify_full_concept_list = [line.rstrip() for line in f.readlines()]
    
    now_time =  datetime.datetime.now()
    formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
    print("{}; Concepts: {:d} ".format(formatted_time,len(modify_full_concept_list)))

05-12-2025 00:53:02; Concepts: 613794 


In [11]:
filter_start_words_len2=['ci','order','resnet','theorem','level','le','ge','class','experiment','pm','figure',
                          'type','dimension','cifar','characteristic','rank','probability','hiv','factor','top',
                        'range','full','mass','central','lemma','ii','pythia','group','gsim']


filter_start_words_len3=['map','galaxy','line','time','observation','tev','emission','transition','mpc','mk','min']


### the start word of concept

filter_concept_start_words=['saving','tracking','measuring','puzzling','injecting','interpreting','gaining','whilst',
                           'inform','control','novel','improving','increase','unexpectedly','detecting','greatly','pm',
                           'accelerate','promote','desired','introducing','decades','canadian','solving','advancing',
                           'performing','showing','via','extract','losing','lose','representing','equiv','set','sets',
                           'playing','play','rightarrow','scenario','ongoing','thorough','conduct','developing','carry',
                           'setting','survey','powerful','development','estimating','corresponding','interesting','place',
                           'never','address','addressing','overcome','overcoming','upcoming','forthcoming','values','neither',
                           'understanding','total','kept','somewhat','somehow','training','proposing','reasonably','regularly',
                           'containing','evolving','accelerating','decreases','read','check','checking','rigorously','reducing',
                           'numerous','encoding','though','models','methods','method','dimensional','body','environmentally',
                            'dimensionally','leveraging','aforementioned','escape','escaping','ethical','noticeable','track',
                           'anomaly','surrounding','analysis','comprehensive','fairly','decays','project','firmly','effect',
                           'showed','improved','maintaining','save','saves','rescue','answers','answering','relativistically',
                           'simulating','outgoing','seen','transform','design','designed','assisting','assist','worlds',
                           'fascinating','became','encourage','encourages','encouraging','findings','infect','infects','sharing',
                           'intervening','langle','end','useful','latter','mathematically','problem','simpler','arguably',
                           'computationally','expensive','technically','insufficient','insufficiently','constructing','repeat',
                           'devices','guaranteed','violent','imposing','identifying','commonly','situation','dependently','papers',
                           'societal','demonstrating','preliminary','test','conducting','deploying','deploy','primary','deepest',
                           'architectural','plays','maps','infer','popular','background','preparing','attracting','attract',
                           'attracts','aroused','measure','motivating','agricultural','putting','focuses','pay','pays','paying',
                           'distribute','distributing','forecasted','capture','munich','existential','biggest','prominent',
                           'supports','spectacular','samples','minimize','evaluating','methodswe','objectivewe','resultswe',
                           'exploring','comparing','complicated','positions','key','capture','captures','irregularly',
                            'surround','remaining']


filter_concept_start_restricted_len=['average','optimal','subject','nontrivial','experimental','individual','large','exact',
                                    'magnitude','expected','common','small','issue','description','empirical','lesssim',
                                     'necessarily','drop','initial','model','environmental','difference','numerical','effective',
                                    'non','modified','continued','enhanced','shortest','longest','throughout','parameters',
                                     'generalized','electrically','extensive','size','simulated','selective','original','big',
                                    'extended','trivial','perfectly','like','complementary','late','topologically','highest',
                                    'human','totally','forward','random','produced','prescribed','impact','compactly','finitely',
                                    'nearest','marked','repeating','defined','principle','widespread','generated','infinite','ancilla',
                                    'third','second','unconventional','particular','greatest','great','actively','lossy','components',
                                    'neighboring','smoothly','subsequent','equivalent','heavily','equivalently','reliable','route',
                                    'stronger','restricted','core','points']

### the last word of concept

filter_concept_last_words=['t_','eta','saving','discus','constitutes','constitute','topic','mood','created','play',
                          'mathematician','near','unique','experimental','carry','repository','generating','depending',
                          'speaker','appearing','collaboration','drop','audience','lie','high','low','higher','lower',
                          'computational','varying','relevant','containing','evolving','training','read','bar','check',
                          'checking','initial','relating','driven','restricted','suffer','originates','originating',
                          'presented','universal','modified','improved','suppressed','corresponding','stronger','large',
                          'coupled','theoretical','dominated','extending','contained','nontrivial','applied','belonging',
                          'belong','belongs','designed','collaborator','coder','alike','practitioners','measured','scattered',
                          'end','fixed','correct','motivated','part','involved','investigator','guarantee','highland',
                          'centric','produced','encountered','supported','represented','aimed','adopted','subtracted',
                          'outside','inside','tend','tends','tended','tending','chooses','detail','concerning','defined',
                          'defines','interact','interacts','interacted','loaded','vanishes','vanish','tendency','admitting',
                          'prior','determined','located','indicated','ass','massive','implication','heated','surrounding',
                          'surrounded','aligned','stemming','characterized']

filter_concept_last_restricted_len=['subject','individual','large','exact','expected','common','small','issue','impact',
                                   'description','numerical','faced','condition','community','reducing','route','visual',
                                   'reduced','agreed','investigation','criterion','knowledge','k','noised','staff','server',
                                   'induced','carried','developing','measuring','thought','water','size','launched','point']

### the whole words of concept

filter_concept_lists=['canada','eastern','shows','mainly','originate','tests','increasingly','ensuring','et','suppress',
                     'enhance','facilitate','tracked','already','exists','exist','updated','governmental','handle','report',
                     'berkeley','considered','considering','crucial','seems','seem','involves','involve','producing','produces',
                     'acting','shed','studying','represents','diverges','reasonable',';\\','\\;','’','managing','exploit','everyday',
                     'raising','||','[[','spent','yr','exceed','choose','meet','meeting','admits','admit','overcomes','known',
                     'went','undergoes','approaches','true','generally','keep','keeps','trains','namely','played','assuming',
                     'assume','assumes','propose','began','begin','grows','grow','growing','ever','enabling','exponentially',
                     'slowly','varies','converges','converge','converging','arbitrarily','contains','evolves','accelerates',
                     'dramatically','substantially','changes','increases','increase','degrade','degrades','alter','abruptly',
                     'guaranteeing','guarantees','sufficient','establishing','corresponds','correspond','implies','relying',
                     'rely','incorporate','incorporates','incorporating','rigorous','severity','schr','ultimate','ultimately',
                     'encode','straightforward','conceptually','still','fail','fails','cycles','around','leverages','leverage',
                     'dinger','introduction','analogue','disappears','disappear','disappearing','vienna','china','usa','japan',
                      'europe','southern','east','west','north','south','french','america','longer','shorter','unseen','publisher',
                     'combines','combine','combining','establishes','conclude','authors','author','research','build','builds',
                     'building','bad','default','v','modify','modifies','whereby','arising','arise','gets','continue','continues',
                     'continuing','highlight','highlights','replaced','getting','challenge','date','overview','maintain','london',
                     'maintains','resetting','reset','upon','publication','elsewhere','journal','article','course','achieving',
                     'reach','saving','maximum','coincide','coincides','coinciding','investigating','generalizing','classroom',
                     'instruction','allowed','conducted','questions','question','acquired','acquires','acquiring','acquire',
                     'simulate','discipline','laboratory','involved','dominate','dominates','involving','expertise','expert',
                      'experts','underlying','guideline','recommended','harm','recommend','trivially','whatever','conducted',
                     'scientist','scientists','interested','anyone','else','readers','reader','engineer','engineers','others',
                     'appear','appearing','appeared','outperform','outperforming','outperformed','amongst','raise','raises',
                     'infecting','publicly','released','share','shares','today','incorrect','incorrectly','health','wish',
                      'alternative','appnl','le','th','supply','uncertain','constructed','certainty','imposed','identified',
                     'identifies','waiting','\\"','tool','equ','contributing','representing','represent','strategies','implemented',
                     'behaviour','comparable','demonstrated','obtains','inefficient','female','male','exclusive','deployed',
                     'outstanding','thesis','develops','obeys','obey','extracted','assign','assigns','assigned','able','interview',
                     'student','prepared','prepare','prepares','teacher','teach','teaching','applications','asserts','assert',
                     'interesting','theoretically','measures','motivation','motivate','motivates','come','comes','coming',
                     'robustness','examine','examines','examining','examined','analyse','analyses','analysed','preventive',
                     'illustristng','immediate','scientific','trusted','exclusively','primarily','asset','always','demanding',
                     'fruitful','idea','captures','observatory','observing','threat','huge','burden','healthcare','smart',
                     'want','win','uescher','terms','sample','drawn','draw','draws','regret','evaluated','evaluates','launch',
                     'analyzed','regions','wit','refines','keeping','ensures','conditions','studies','suggested','sampled',
                     'exhibits','exhibited','redundant','remained','infrastructure','treated','treat','treating','cache','align'] 



filter_concept_restricted_len=['north','western','see','saving','decrease','influence','question','simple','specific','essential',
                              'ranked','answer','twitter','negligible','challenge','address','addressing','wrong','total','local',
                              'high','low','higher','lower','largest','smallest','fast','slow','varying','altered','sharply',
                              'direction','contribution','bar','missing','critical','critically','intrinsically','global','min',
                              'environment','methodology','major','body','prescription','setting','basic','dataset','datasets',
                              'theoretical','experimental','universal','near','proof','proofs','facility','actor','policy','cost'
                              'conventional','suppressed','data','time','online','obstacle','avoiding','avoids','modern','strictly',
                              'separate','separates','separating','minimally','minimal','maximum','maximally','maximal','spatially',
                              'direct','privacy','parameter','resonantly','traditional','validation','weakly','sub','access','test',
                              'age','across','frame','throughput','survey','independent','little','h','item','injury','distinct',
                              'key','world','working','accessible','shared','merger','motivated','change','sharing','event','rate',
                              'temperature','device','right','factor','joint','understanding','fashion','way','ways','activity',
                              'related','bound','flat','histogram','regular','implementation','positive','realistic','efficient',
                              'supported','normal','cover','type','types','enough','strategy','adopted','increased','controlled',
                              'real','complex','player','region','architecture','driving','control','development','repeated','role',
                              'design','interacting','assignment','observer','initially','application','interest','measure','position',
                              'science','iteration','benchmarking','epidemic','area','viable','platform','flexible','domain',
                              'assumed','trust','specifically','demand','classical','processing','forecasting','capture','database',
                              'consistent','estimation','rising','prior','term','infinitely','determined','de','accuracy','prediction',
                              'intermediate','strong','young','level','pathway','relationship','monitoring','enabled','alignment',
                              'aligned','larger','limiting']



### others

filter_contain_str_lists=['meter telescope','sigma significance','derivative term','small enough','large enough','sigma level',
                          'limit set','upper limit','large range','small range','time interval','web site','open research',
                         'realistic device','large device','small device','lower limit','public health','building block',
                         'realistic model','real model','real life','world health','human health','public health','major issue',
                         'open issue','open problem','long standing','high level','low level','peer reviewed','computational cost',
                         'open access','open question','negative answer','systematic numerical','careful analysis','high statistical',
                          'numerical analysis','low quality','general dependence','large database','real word','effective method',
                         'computational pipeline','strings ending','non trivial','general high','took place','exact solution',
                         'explicit solution','high impact','numerical study','theoretical study','cost path','order term',
                         'control parameter','weight loss','initial condition','high area','large area','small area','high accuracy',
                          'high risk','particular condition','particular attention','event related','average level','key goal',
                         'higher accuracy','highest accuracy','accurate theoretical','training dataset','key step','initial step',
                         'strong solution','unique global','key feature','small limit','large limit']


filter_contain_str=['low cost','high cost','n x n','*',':','+',']',';','throughput search']


filter_concept_second_words=['experimental','challenge','data']

    
filter_concept_low_score_words=['underlying','normally','necessarily','usual','expected','efficient','relevant','strongly','realistic',
                                'normal','simple','major','basic','surrounding','numerical','specific','experimental','conventional',
                               'modified','modern','strictly','generalized','traditional','original','nontrivial','totally','particular',
                                'increased','popular','initially','viable','flexible','unconventional','conventional','assumed','larger',
                               'specifically','actively','smoothly','subsequent','reliable','initial','stronger','highest']

filter_concept_low_score_words2=['model','method','technique','application','approach','community','framework','research',
                                 'pipeline','literature','strategy','workflow','paradigm','tool','architecture',
                                 'technology','methodology','prescription','pathway']

number_char_list=['0','1','2','3','4','5','6','7','8','9']
number_word_list=['one','two','three','four','five','six','seven','eight','nine','ten','eleven','twelve']

       
modify_improved_concept_list=[]
    
for one_concept in modify_full_concept_list:
    
    separated_words=one_concept.split()
    do_remove=0
    
    for word in separated_words:
        if word in number_word_list:
            do_remove=1
            break
        
        if word in filter_concept_lists: ## check whether contains to-remove words
            do_remove=1
            break
        if is_valid_year(word): ## check whether it conatins a number as year
            do_remove=1
            break  
          
    if do_remove ==0:
        for x in number_char_list:
            if x in one_concept:
                do_remove=1
                break 
    
    for ii in range(len(filter_contain_str)):  ## check whether it contains a certain string
        if filter_contain_str[ii] in one_concept: 
            do_remove=1
            break
    
    for filter_concepts in filter_contain_str_lists: ## check contain any of the two words
        seperated_filter=filter_concepts.split()
        counts=0
        for word_filter in seperated_filter:
            
            if word_filter in separated_words: 
                counts+=1
            if counts==len(seperated_filter):
                do_remove=1
                break
                
    if do_remove==0:   
        
        if len(separated_words)==2:  ## fix the length==2
            for word in separated_words:
                if word in filter_concept_restricted_len: #check whether it cantains to-remove words
                    do_remove=1
                    break

            if separated_words[0] in filter_concept_start_restricted_len: #check the start word 
                do_remove=1
            if separated_words[-1] in filter_concept_last_restricted_len: #check the last word 
                do_remove=1

            # the first is a word, and the rest is a number
            if separated_words[1].isnumeric() and separated_words[0] in filter_start_words_len2: 
                do_remove=1 
                
            if (len(separated_words[0])!=1 and len(separated_words[1])==1) or (len(separated_words[0])==1 and len(separated_words[1])!=1):
                do_remove=1  


        if len(separated_words)==3: ## fix the length==3
            # the first two are number and the rest is a word   
            if separated_words[0].isnumeric() and  separated_words[1].isnumeric() and separated_words[2] in filter_start_words_len3:
                do_remove=1 
                
            if len(separated_words[0])!=1 and len(separated_words[1])==1 and len(separated_words[2])==1:
                do_remove=1 
            if len(separated_words[0])==1 and len(separated_words[1])!=1 and len(separated_words[2])==1:
                do_remove=1    
            if len(separated_words[0])!=1 and len(separated_words[1])==1 and len(separated_words[2])==1:
                do_remove=1    
            if len(separated_words[0])!=1 and len(separated_words[1])!=1 and len(separated_words[2])==1:
                do_remove=1
            if len(separated_words[0])!=1 and len(separated_words[1])==1 and len(separated_words[2])!=1:
                do_remove=1
    
        #check the start word; no length restriction 
        if separated_words[0] in filter_concept_start_words:
            do_remove=1

         #check the last word; no length restriction   
        if separated_words[-1] in filter_concept_last_words:
            do_remove=1

        if len(separated_words)%2==0:
            half0=separated_words[0:int(len(separated_words)/2)]
            half1=separated_words[int(len(separated_words)/2):int(len(separated_words))]
            if half0[-1][-1]=='s':
                half0[-1] = half0[-1][:-1]
            if half1[-1][-1]=='s':
                half1[-1] = half1[-1][:-1]
            if half0 == half1:
                 do_remove=1
            
    if do_remove==0:
        
        if len(separated_words)>=3:
            if separated_words[1] in filter_concept_second_words:  #check the second word
                do_remove=1

            ## filter low score words 
            if separated_words[0] in filter_concept_low_score_words:  

                the_rest_words  = separated_words[1:len(separated_words)]
                no_low_words_concept=' '.join(the_rest_words)

                if no_low_words_concept in modify_full_concept_list:
                    do_remove=1

            if separated_words[-1] in filter_concept_low_score_words2:
                the_previous_words  = separated_words[0:len(separated_words)-1]
                no_low_words_concept=' '.join(the_previous_words)
                if no_low_words_concept in modify_full_concept_list:
                    do_remove=1

        if separated_words[-1][-1]=="s": ## plural
            plural_string = one_concept[:-1]
            if plural_string in modify_full_concept_list:
                do_remove=1
            
        if do_remove==0:
            modify_improved_concept_list.append(one_concept)
        
print("Concepts: {:d} ; Store: {:d}; Remove: {:d} ".format(len(modify_full_concept_list), len(modify_improved_concept_list),len(modify_full_concept_list)-len(modify_improved_concept_list)))
elapsed_time = time.time() - starting_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

print('\n')
now_time =  datetime.datetime.now()
formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
print("{}; Concepts: {:d} ".format(formatted_time,len(modify_improved_concept_list)))

Concepts: 613794 ; Store: 434771; Remove: 179023 
Elapsed time: 315.98 seconds


05-12-2025 00:56:17; Concepts: 434771 


输出文件
full_concepts_stored_lists_6_9_improved_modify.txt：包含所有经过完整清洗流程的最终关键词列表

In [12]:
if os.path.exists("full_concepts_stored_lists_6_9_improved_modify.txt"):
    os.remove("full_concepts_stored_lists_6_9_improved_modify.txt")
    print("delete the original txt file")
    
    f = open("full_concepts_stored_lists_6_9_improved_modify.txt", "a")
    for ii in range(len(modify_improved_concept_list)):
        f.write(modify_improved_concept_list[ii]+'\n')
    f.close()
    print("create a new txt file.")
    
else:
    f = open("full_concepts_stored_lists_6_9_improved_modify.txt", "a")
    for ii in range(len(modify_improved_concept_list)):
        f.write(modify_improved_concept_list[ii]+'\n')
    f.close()
    print("create text and store information.")
    
now_time =  datetime.datetime.now()
formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
print("{}; Concepts: {:d} ".format(formatted_time,len(modify_improved_concept_list)))

delete the original txt file
create a new txt file.
05-12-2025 00:58:30; Concepts: 434771 


In [13]:
## just a backup of the full concepts
with open("full_concept_list.pkl", "wb") as output_file:
    pickle.dump(modify_improved_concept_list, output_file)

In [14]:
import pickle

# 加载 .pkl 文件
with open("full_concept_list.pkl", "rb") as input_file:
    full_concept_list = pickle.load(input_file)

# 将关键词写入 .txt 文件
with open("full_concept_list.txt", "w") as f:
    for concept in full_concept_list:
        f.write(concept + "\n")

print("Concepts saved to full_concept_list.txt")


Concepts saved to full_concept_list.txt
